In [ ]:
from pathlib import Path
import config as cfg
from WSI_active_learning import main

CLASSES = ['D', 'M', 'N']

#PATCH_STRATEGY = ['passive', 'random','confidence', 'entropy', 'coreset', 'badge'] 
PATCH_STRATEGY = ['confidence'] 

WSI_PATCH_SCORE = ["random", "confidence", "entropy", "cdal", "class_conf"] 
#WSI_PATCH_SCORE = ['class_conf'] # 'entropy', 'confidence', 'class_conf', 'cdal' 

WSI_REP_METHOD = ["random", "average", "cdal", "kl_dis_x", "kl_rep", "kl_rank", "js_dis_x", "js_rep", "js_rank"]
#WSI_REP_METHOD = ['kl_rep'] 


NUM_RUNS = 1    # RUN THE CODE K TIMES


if __name__=='__main__':  
    for _ in range(NUM_RUNS):
        trial_number = 1

        while Path(f"{cfg.BASE_DIR}{trial_number}").exists():
            trial_number += 1
        log_dir = f"{cfg.BASE_DIR}{trial_number}"
        Path(log_dir).mkdir(exist_ok=False, parents=True)
        print(f"Log dir: {log_dir}")
        
        for wsi_rep_method in WSI_REP_METHOD : 
            for wsi_patch_score in WSI_PATCH_SCORE:
                for patch_strategy in PATCH_STRATEGY : 

                    if wsi_patch_score == "random" : 
                        if wsi_rep_method != "random" : continue 

                    elif wsi_patch_score in ["confidence", "entropy"] : 
                        if wsi_rep_method != 'average' : continue 

                    elif wsi_patch_score == "cdal" : 
                        if wsi_rep_method != 'cdal' : continue 
                    
                    else : 
                        if wsi_rep_method not in ["kl_dis_x", "kl_rep", "kl_rank", "js_dis_x", "js_rep", "js_rank"] : continue


                    logger_name = f"{wsi_patch_score}_{wsi_rep_method}_{patch_strategy}_{cfg.INITIAL_WSI_PER_CLASS}WSI"
                    log_file = f"{log_dir}/test_{cfg.MODEL_NAME}_{logger_name}.txt"
                    

                    print(f"{wsi_patch_score}, {patch_strategy}, {cfg.INITIAL_WSI_PER_CLASS} initial wsi per class, {cfg.NUM_WSI_PER_GENERATION} wsi per generation, {cfg.NUM_PATCHES_PER_WSI} patches per wsi")
                    with open(log_file, "a") as logger:
                        logger.write("cycle, number_of_wsi,number_of_patches_per_wsi, train_patches, time, auroc_score, accuracy\n")
                        main(cfg= cfg,
                            patch_strategy=patch_strategy,
                            wsi_patch_score = wsi_patch_score, 
                            wsi_rep_method = wsi_rep_method,
                            logger=logger) 



    